This notebook performs zero-shot and few-shot learning on the off-the-shelf LLMs, leading to the result shown in Section 5.2.1.

This notebook is inspired by: https://github.com/unslothai/unsloth

# Packages

In [ ]:
%%writefile requirements.txt
unsloth==2025.6.2
unsloth_zoo==2025.6.1
trl==0.15.2
xformers==0.0.29.post3
bitsandbytes==0.46.0
peft==0.15.2
accelerate==1.7.0
torch==2.6.0+cu124
transformers==4.52.4
datasets==3.6.0
pandas==2.2.2
scikit-learn==1.6.1
sentencepiece==0.2.0
huggingface-hub
hf_transfer

In [ ]:
%%capture
!pip install --no-cache-dir -r requirements.txt

from unsloth import FastLanguageModel
import torch
import pandas as pd
import numpy as np
from datasets import Dataset
import xml.etree.ElementTree as ET
import torch.nn.functional as F
import ast
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Functions

Load data

In [ ]:
def load_data(file_path):
    """
    Load and preprocess the XML file into a pandas DataFrame.
    """

    # Load and parse XML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Extract data into a list of dictionaries
    data = []

    # Iterate through the XML structure and extract infomation from the reviews
    for review in root.findall('Review'):
      review_id = review.get('rid')

      for sentence in review.findall('.//sentence'):
        text = sentence.find('text').text

        # Show nothing rather than 'NULL' in the prompt
        for opinion in sentence.findall('.//Opinion'):
          aspect = opinion.get('target')
          if aspect == 'NULL':
            aspect = ''

          # Adjust the categories to be more informative for the prompt
          category = opinion.get('category').lower().replace('#', ' ').replace('_',' and ')
          if category == 'food general':
              category = 'food style and options'
          elif category == 'service general':
              category = 'service'
          elif category == 'restaurant general' or category == 'restaurant miscellaneous':
              category = 'restaurant'
          elif category == 'ambience general':
              category = 'ambience'
          elif category == 'location general':
              category = 'location'

          # Represent the aspect as 'term (category entity)'
          aspect_term_category = aspect + ' (' + category + ')'

          # Format the demonstrations beforehand to easier format the few-shot prompts later on
          demonstration = 'Sentence: ' + text + '\n' + 'Aspect: ' + aspect_term_category + '\n' + 'Sentiment: ' + opinion.get('polarity') + '\n'

          data.append({
              "sentence": text,
              "aspect": aspect,
              "category": category,
              "aspect_term_category": aspect_term_category,
              "demonstration": demonstration,
              "sentiment": opinion.get('polarity')
          })

    # Convert to DataFrame
    df = pd.DataFrame(data)
    y = df['sentiment'].values
    label_to_idx = {"negative": 0, "neutral": 1, "positive": 2}
    df['sentiment_label'] = df['sentiment'].map(label_to_idx)

    # Return both the features DataFrame and the sentiment labels
    return df, y

Classification of zero-shot prompt

In [ ]:
def zero_shot_classification(dataset):
    """
    Perform zero-shot classification by predicting the most likely sentiment.
    """

    # Define the classes and get their token IDS
    classes = ["negative", "neutral", "positive"]
    class_token_ids = [tokenizer.encode(c, add_special_tokens=False)[0] for c in classes]

    # Create lists to store the predicted classes
    pred_sent = []

    # Iterate over the dataset
    for row in tqdm(dataset):
      input = row['zero_shot_prompt']
      inputs = tokenizer(input, return_tensors="pt").to('cuda')

      # Perform inference without computing gradients to save memory
      with torch.no_grad():
        # This returns the outputs for all input tokens and the next one
        outputs = model(**inputs)

      # The predicted class should be at the last token (= logit.size(1) -1)
      logits = outputs.logits
      prediction_position = logits.size(1) - 1

      # Retrieve the logits for the classes at the specified position (end of prompt)
      logits_for_prediction = logits[:, prediction_position, :]
      class_logits = logits_for_prediction[:, class_token_ids]

      # Obtain the predicted probabilities class by using the softmax and argmax function respectively
      class_probs = F.softmax(class_logits, dim=-1)
      probs = class_probs.cpu().detach().float().numpy()
      pred_sent.append(classes[np.argmax(probs)])

      # To save memory
      del inputs, outputs, logits, logits_for_prediction, class_logits, class_probs
      torch.cuda.empty_cache()

    return pred_sent

Classification for few-shot prompt ()

In [ ]:
def few_shot_classification(dataset):
    """
    Perform few-shot classification by predicting the most likely sentiment.
    """

    # Define the classes and get their token IDS
    classes = ["negative", "neutral", "positive"]
    class_token_ids = [tokenizer.encode(c, add_special_tokens=False)[0] for c in classes]

    # Create lists to store the predicted classes
    pred_sent = []

    # Iterate over the dataset
    for row in tqdm(dataset):
      input = row['few_shot_prompt'] # Few-shot prompt rather than zero-shot prompt
      inputs = tokenizer(input, return_tensors="pt").to('cuda')

      # Perform inference without computing gradients to save memory
      with torch.no_grad():
        # This returns the outputs for all input tokens and the next one
        outputs = model(**inputs)

      # The predicted class should be at the last token (= logit.size(1) -1)
      logits = outputs.logits
      prediction_position = logits.size(1) - 1

      # Retrieve the logits for the classes at the specified position (end of prompt)
      logits_for_prediction = logits[:, prediction_position, :]
      class_logits = logits_for_prediction[:, class_token_ids]

      # Obtain the predicted probabilities class by using the softmax and argmax function respectively
      class_probs = F.softmax(class_logits, dim=-1)
      probs = class_probs.cpu().detach().float().numpy()
      pred_sent.append(classes[np.argmax(probs)])

      # To save memory
      del inputs, outputs, logits, logits_for_prediction, class_logits, class_probs
      torch.cuda.empty_cache()

    return pred_sent

In [ ]:
def run_few_shot_preds(dataset_train, dataset_test, dem_selection, dem_order, k, order_seed, selection_seed):
    """
    This function allows us to perform few-shot classifation with different input arguments after one another, without having to reinitialize the data.
    Without this function, we would get the error that the column few_shot_prompt is already in the dataset if we tried to format the few-shot prompts again for a different configuration.
    """

    few_shot_dict = format_few_shot_prompts(dataset_train, dataset_test, dem_selection, dem_order, k, order_seed, selection_seed)
    if "few_shot_prompt" in dataset_test.column_names:
        dataset_test = dataset_test.remove_columns("few_shot_prompt")
    dataset_test = dataset_test.add_column(
        "few_shot_prompt",
        few_shot_dict["few_shot_prompt"]
    )

    y_pred_few = few_shot_classification(dataset_test)
    return y_pred_few

# Prompts

Load either the prompts for the Mistral model, or the prompts for the Llama and Gemma model

## Mistral

In [ ]:
zero_shot_prompt = """Classify the sentiment expressed towards the given aspect within the provided sentence as 'negative', 'neutral' or 'positive'.

### Sentence:
{}

### Aspect:
{}

### Sentiment:
 """

def format_zero_shot_prompts(data):
    """
    This function formats the prompts for zero-shot classification.
    """

    sentences      = data["sentence"]
    aspects        = data["aspect_term_category"]
    prompts = []

    # Format the zero-shot prompt for every test instance
    for sentence, aspect in zip(sentences, aspects):
      prompt = zero_shot_prompt.format(sentence, aspect)
      prompts.append(prompt)

    return { "zero_shot_prompt" : prompts }

In [ ]:
few_shot_prompt = """Classify the sentiment expressed towards the given aspect within the provided sentence as 'negative', 'neutral' or 'positive'.

### Examples:
{}
### Sentence:
{}

### Aspect:
{}

Sentiment:
 """

def format_examples(subset, dem_order, order_seed):
    """
    This function orders the demonstrations shown within the prompt
    """

    # Order the demonstrations, given the ordering method
    if dem_order == 'sso':
      ordered_set = subset
    elif dem_order == 'ro':
      ordered_set = subset.shuffle(seed=order_seed)
    elif dem_order == 'swn':
      ordered_set = subset.sort('swn')
    elif dem_order == 'sl':
      ordered_set = subset.sort('sl')
    elif dem_order == 'ce':
      ordered_set = subset.sort('ce')

    # Print the ordered demonstrations below each other
    k_examples = ''
    for row in ordered_set:
      k_examples += row['demonstration']
      k_examples += '\n'

    return k_examples


def format_few_shot_prompts(train_data, test_data, dem_selection, dem_order, k, order_seed, selection_seed):
    """
    This function formats the prompts for few-shot classification.
    """

    # Initialize all the elements needed to form the prompts
    np.random.seed(selection_seed)
    sentences      = test_data["sentence"]
    aspects        = test_data["aspect_term_category"]
    indices       = test_data["original_indices"]
    prompts = []

    # For every test instance
    for sentence, aspect, index in zip(sentences, aspects, indices):

      # Select the demonstrations
      if dem_selection == 'rs':
        k_indices = np.random.choice(len(train_data), size=k, replace=False)
      elif dem_selection == 'hkate':
        k_indices = test_data['HKATE_indices'][index][0:k]
        if dem_order == 'sso':
          k_indices = k_indices[::-1]
      elif dem_selection == 'kate':
        k_indices = test_data['KATE_indices'][index][0:k]
        if dem_order == 'sso':
          k_indices = k_indices[::-1]

      # Order and format the demonstrations
      k_examples = format_examples(train_data.select(k_indices), dem_order, order_seed)

      # Format the few-shot prompt
      prompt = few_shot_prompt.format(k_examples, sentence, aspect)
      prompts.append(prompt)

    return { "few_shot_prompt" : prompts }

## Llama and Gemma

In [ ]:
zero_shot_prompt = """Classify the sentiment expressed towards the given aspect within the provided sentence as 'negative', 'neutral' or 'positive'.

### Sentence:
{}

### Aspect:
{}

### Sentiment:
"""

def format_zero_shot_prompts(data):
    """
    This function formats the prompts for zero-shot classification.
    """

    sentences      = data["sentence"]
    aspects        = data["aspect_term_category"]
    prompts = []

    # Format the zero-shot prompt for every test instance
    for sentence, aspect in zip(sentences, aspects):
      prompt = zero_shot_prompt.format(sentence, aspect)
      prompts.append(prompt)

    return { "zero_shot_prompt" : prompts }

In [ ]:
few_shot_prompt = """Classify the sentiment expressed towards the given aspect within the provided sentence as 'negative', 'neutral' or 'positive'.

### Examples:
{}
### Sentence:
{}

### Aspect:
{}

Sentiment:
"""

def format_examples(subset, dem_order, order_seed):
    """
    This function orders the demonstrations shown within the prompt
    """

    # Order the demonstrations, given the ordering method
    if dem_order == 'sso':
      ordered_set = subset
    elif dem_order == 'ro':
      ordered_set = subset.shuffle(seed=order_seed)
    elif dem_order == 'swn':
      ordered_set = subset.sort('swn')
    elif dem_order == 'sl':
      ordered_set = subset.sort('sl')
    elif dem_order == 'ce':
      ordered_set = subset.sort('ce')

    # Print the ordered demonstrations below each other
    k_examples = ''
    for row in ordered_set:
      k_examples += row['demonstration']
      k_examples += '\n'

    return k_examples


def format_few_shot_prompts(train_data, test_data, dem_selection, dem_order, k, order_seed, selection_seed):
    """
    This function formats the prompts for few-shot classification.
    """

    # Initialize all the elements needed to form the prompts
    np.random.seed(selection_seed)
    sentences      = test_data["sentence"]
    aspects        = test_data["aspect_term_category"]
    indices       = test_data["original_indices"]
    prompts = []

    # For every test instance
    for sentence, aspect, index in zip(sentences, aspects, indices):

      # Select the demonstrations
      if dem_selection == 'rs':
        k_indices = np.random.choice(len(train_data), size=k, replace=False)
      elif dem_selection == 'hkate':
        k_indices = test_data['HKATE_indices'][index][0:k]
        if dem_order == 'sso':
          k_indices = k_indices[::-1]
      elif dem_selection == 'kate':
        k_indices = test_data['KATE_indices'][index][0:k]
        if dem_order == 'sso':
          k_indices = k_indices[::-1]

      # Order and format the demonstrations
      k_examples = format_examples(train_data.select(k_indices), dem_order, order_seed)

      # Format the few-shot prompt
      prompt = few_shot_prompt.format(k_examples, sentence, aspect)
      prompts.append(prompt)

    return { "few_shot_prompt" : prompts }

# Main

To run this code, first upload the files:

*   '201x_Restaurants_Train.xml'
*   '201x_Restaurants_Test.xml'
*   'top_k_indices_201x.csv'
*   'swn_complexity_scores_201x.csv'
*   'sentence_lengths_201x.csv'
*   'ce_model_201x.csv'

with model being either mistral, llama or gemma and 201x being either 2015 or 2016.

NOTE: Make sure to only (or at last) run the prompts corresponding to the model you are using! Otherwise, inconsistency in sentiment label tokenization will mess up the results!

Once you have completed all the runs for a model, reconnect the runtime before moving on to another LLM.

## Load the model

Choose and load a model

In [ ]:
#model_name = "unsloth/mistral-7b-v0.3-bnb-4bit"
model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit"
#model_name = "unsloth/gemma-2-9b-bnb-4bit"

In [ ]:
max_seq_length = 1024
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 1024,
    dtype = None,
    load_in_4bit = True,
)

## Load data

Load either the 2015 or 2016 data

### 2015

In [ ]:
# Load train data
df_train, y_train = load_data('2015_Restaurants_Train.xml')
df_train['original_indices'] = df_train.index
df_train['sl'] = pd.read_csv('sentence_lengths_2015.csv')
df_train['swn'] = pd.read_csv('swn_complexity_scores_2015.csv')
if model_name == "unsloth/Meta-Llama-3.1-8B-bnb-4bit":
  df_train['ce'] = pd.read_csv('ce_llama_2015.csv')
elif model_name == "unsloth/mistral-7b-v0.3-bnb-4bit":
  df_train['ce'] = pd.read_csv('ce_mistral_2015.csv')
elif model_name == "unsloth/gemma-2-9b-bnb-4bit":
  df_train['ce'] = pd.read_csv('ce_gemma_2015.csv')

# Load test data
df_test, y_test = load_data('2015_Restaurants_Test.xml')
df_test['original_indices'] = df_test.index
df_test['KATE_indices'] = pd.read_csv('top_k_indices_2015.csv')
df_test['KATE_indices'] = df_test['KATE_indices'].apply(ast.literal_eval)

# Convert to huggingface Datasets
dataset_train = Dataset.from_pandas(df_train)
dataset_test = Dataset.from_pandas(df_test)

### 2016

In [ ]:
# Load train data
df_train, y_train = load_data('2016_Restaurants_Train.xml')
df_train['original_indices'] = df_train.index
df_train['sl'] = pd.read_csv('sentence_lengths_2016.csv')
df_train['swn'] = pd.read_csv('swn_complexity_scores_2016.csv')
if model_name == "unsloth/Meta-Llama-3.1-8B-bnb-4bit":
  df_train['ce'] = pd.read_csv('ce_llama_2016.csv')
elif model_name == "unsloth/mistral-7b-v0.3-bnb-4bit":
  df_train['ce'] = pd.read_csv('ce_mistral_2016.csv')
elif model_name == "unsloth/gemma-2-9b-bnb-4bit":
  df_train['ce'] = pd.read_csv('ce_gemma_2016.csv')

# Load test data
df_test, y_test = load_data('2016_Restaurants_Test.xml')
df_test['original_indices'] = df_test.index
df_test['KATE_indices'] = pd.read_csv('top_k_indices_2016.csv')
df_test['KATE_indices'] = df_test['KATE_indices'].apply(ast.literal_eval)

# Convert to huggingface Datasets
dataset_train = Dataset.from_pandas(df_train)
dataset_test = Dataset.from_pandas(df_test)

## Zero-Shot Learning

Perform zero-shot learning, print the evaluation metrics, and save the predictions

In [ ]:
dataset_test = dataset_test.map(format_zero_shot_prompts, batched = True,)
y_pred_zero = zero_shot_classification(dataset_test)
print('\n zero-shot: \n', classification_report(y_test, y_pred_zero, digits = 4))
pd.Series(y_pred_zero).to_csv('llama2016_zero.csv', index=False)

## Few-Shot Learning

Perform few-shot learning, print the evaluation metrics, and save the predictions for all configurations.

Here k are the number of demonstrations and the abbreviations are defined as:

*   rs = Random Selection
*   kate = K-nn Augmented in-conText Example selection
*   hkate = Hierachical KATE
*   ro = Random Ordering
*   sso = Semantic Similarity Ordering
*   sl = Sentence Length
*   swn = SentiWordNet
*   ce = Cross-Entropy



### k = 5, RS

In [ ]:
y_pred_k5_rs_ro1 = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='rs', dem_order = 'ro', k = 5, order_seed = 1, selection_seed = 1)
print('\n 5-shot rs ro 1: \n', classification_report(y_test, y_pred_k5_rs_ro1, digits = 4))
pd.Series(y_pred_k5_rs_ro1).to_csv('llama2016_k5_rs_ro1.csv', index=False)

In [ ]:
y_pred_k5_rs_ro2 = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='rs', dem_order = 'ro', k = 5, order_seed = 2, selection_seed = 1)
print('\n 5-shot rs ro 2: \n', classification_report(y_test, y_pred_k5_rs_ro2, digits = 4))
pd.Series(y_pred_k5_rs_ro2).to_csv('llama2016_k5_rs_ro2.csv', index=False)

In [ ]:
y_pred_k5_rs_ro3 = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='rs', dem_order = 'ro', k = 5, order_seed = 3, selection_seed = 1)
print('\n 5-shot rs ro 3: \n', classification_report(y_test, y_pred_k5_rs_ro3, digits = 4))
pd.Series(y_pred_k5_rs_ro3).to_csv('llama2016_k5_rs_ro3.csv', index=False)

In [ ]:
y_pred_k5_rs_sl = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='rs', dem_order = 'sl', k = 5, order_seed = 1, selection_seed = 1)
print('\n 5-shot rs sl: \n', classification_report(y_test, y_pred_k5_rs_sl, digits = 4))
pd.Series(y_pred_k5_rs_sl).to_csv('llama2016_k5_rs_sl.csv', index=False)

In [ ]:
y_pred_k5_rs_swn = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='rs', dem_order = 'swn', k = 5, order_seed = 1, selection_seed = 1)
print('\n 5-shot rs swn: \n', classification_report(y_test, y_pred_k5_rs_swn, digits = 4))
pd.Series(y_pred_k5_rs_swn).to_csv('llama2016_k5_rs_swn.csv', index=False)

In [ ]:
y_pred_k5_rs_ce = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='rs', dem_order = 'ce', k = 5, order_seed = 1, selection_seed = 1)
print('\n 5-shot rs ce: \n', classification_report(y_test, y_pred_k5_rs_ce, digits = 4))
pd.Series(y_pred_k5_rs_ce).to_csv('llama2016_k5_rs_ce.csv', index=False)

### K = 5, KATE

In [ ]:
y_pred_k5_kate_ro1 = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='kate', dem_order = 'ro', k = 5, order_seed = 1, selection_seed = 1)
print('\n 5-shot kate ro 1: \n', classification_report(y_test, y_pred_k5_kate_ro1, digits = 4))
pd.Series(y_pred_k5_kate_ro1).to_csv('llama2016_k5_kate_ro1.csv', index=False)

In [ ]:
y_pred_k5_kate_ro2 = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='kate', dem_order = 'ro', k = 5, order_seed = 2, selection_seed = 1)
print('\n 5-shot kate ro 2: \n', classification_report(y_test, y_pred_k5_kate_ro2, digits = 4))
pd.Series(y_pred_k5_kate_ro2).to_csv('llama2016_k5_kate_ro2.csv', index=False)

In [ ]:
y_pred_k5_kate_ro3 = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='kate', dem_order = 'ro', k = 5, order_seed = 3, selection_seed = 1)
print('\n 5-shot kate ro 3: \n', classification_report(y_test, y_pred_k5_kate_ro3, digits = 4))
pd.Series(y_pred_k5_kate_ro3).to_csv('llama2016_k5_kate_ro3.csv', index=False)

In [ ]:
y_pred_k5_kate_sso = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='kate', dem_order = 'sso', k = 5, order_seed = 1, selection_seed = 1)
print('\n 5-shot kate sso: \n', classification_report(y_test, y_pred_k5_kate_sso, digits = 4))
pd.Series(y_pred_k5_kate_sso).to_csv('llama2016_k5_kate_sso.csv', index=False)

In [ ]:
y_pred_k5_kate_sl = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='kate', dem_order = 'sl', k = 5, order_seed = 1, selection_seed = 1)
print('\n 5-shot kate sl: \n', classification_report(y_test, y_pred_k5_kate_sl, digits = 4))
pd.Series(y_pred_k5_kate_sl).to_csv('llama2016_k5_kate_sl.csv', index=False)

In [ ]:
y_pred_k5_kate_swn = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='kate', dem_order = 'swn', k = 5, order_seed = 1, selection_seed = 1)
print('\n 5-shot kate swn: \n', classification_report(y_test, y_pred_k5_kate_swn, digits = 4))
pd.Series(y_pred_k5_kate_swn).to_csv('llama2016_k5_kate_swn.csv', index=False)

In [ ]:
y_pred_k5_kate_ce = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='kate', dem_order = 'ce', k = 5, order_seed = 1, selection_seed = 1)
print('\n 5-shot kate ce: \n', classification_report(y_test, y_pred_k5_kate_ce, digits = 4))
pd.Series(y_pred_k5_kate_ce).to_csv('llama2016_k5_kate_ce.csv', index=False)

### K = 10, RS

In [ ]:
y_pred_k10_rs_ro1 = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='rs', dem_order = 'ro', k = 10, order_seed = 1, selection_seed = 1)
print('\n 10-shot rs ro 1: \n', classification_report(y_test, y_pred_k10_rs_ro1, digits = 4))
pd.Series(y_pred_k10_rs_ro1).to_csv('llama2016_k10_rs_ro1.csv', index=False)

In [ ]:
y_pred_k10_rs_ro2 = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='rs', dem_order = 'ro', k = 10, order_seed = 2, selection_seed = 1)
print('\n 10-shot rs ro 2: \n', classification_report(y_test, y_pred_k10_rs_ro2, digits = 4))
pd.Series(y_pred_k10_rs_ro2).to_csv('llama2016_k10_rs_ro2.csv', index=False)

In [ ]:
y_pred_k10_rs_ro3 = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='rs', dem_order = 'ro', k = 10, order_seed = 3, selection_seed = 1)
print('\n 10-shot rs ro 3: \n', classification_report(y_test, y_pred_k10_rs_ro3, digits = 4))
pd.Series(y_pred_k10_rs_ro3).to_csv('llama2016_k10_rs_ro3.csv', index=False)

In [ ]:
y_pred_k10_rs_sl = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='rs', dem_order = 'sl', k = 10, order_seed = 1, selection_seed = 1)
print('\n 10-shot rs sl: \n', classification_report(y_test, y_pred_k10_rs_sl, digits = 4))
pd.Series(y_pred_k10_rs_sl).to_csv('llama2016_k10_rs_sl.csv', index=False)

In [ ]:
y_pred_k10_rs_swn = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='rs', dem_order = 'swn', k = 10, order_seed = 1, selection_seed = 1)
print('\n 10-shot rs swn: \n', classification_report(y_test, y_pred_k10_rs_swn, digits = 4))
pd.Series(y_pred_k10_rs_swn).to_csv('llama2016_k10_rs_swn.csv', index=False)

In [ ]:
y_pred_k10_rs_ce = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='rs', dem_order = 'ce', k = 10, order_seed = 1, selection_seed = 1)
print('\n 10-shot rs ce: \n', classification_report(y_test, y_pred_k10_rs_ce, digits = 4))
pd.Series(y_pred_k10_rs_ce).to_csv('llama2016_k10_rs_ce.csv', index=False)

### K = 10, KATE

In [ ]:
y_pred_k10_kate_ro1 = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='kate', dem_order = 'ro', k = 10, order_seed = 1, selection_seed = 1)
print('\n 10-shot kate ro 1: \n', classification_report(y_test, y_pred_k10_kate_ro1, digits = 4))
pd.Series(y_pred_k10_kate_ro1).to_csv('llama2016_k10_kate_ro1.csv', index=False)

In [ ]:
y_pred_k10_kate_ro2 = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='kate', dem_order = 'ro', k = 10, order_seed = 2, selection_seed = 1)
print('\n 10-shot kate ro 2: \n', classification_report(y_test, y_pred_k10_kate_ro2, digits = 4))
pd.Series(y_pred_k10_kate_ro2).to_csv('llama2016_k10_kate_ro2.csv', index=False)

In [ ]:
y_pred_k10_kate_ro3 = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='kate', dem_order = 'ro', k = 10, order_seed = 3, selection_seed = 1)
print('\n 10-shot kate ro 3: \n', classification_report(y_test, y_pred_k10_kate_ro3, digits = 4))
pd.Series(y_pred_k10_kate_ro3).to_csv('llama2016_k10_kate_ro3.csv', index=False)

In [ ]:
y_pred_k10_kate_sso = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='kate', dem_order = 'sso', k = 10, order_seed = 1, selection_seed = 1)
print('\n 10-shot kate sso: \n', classification_report(y_test, y_pred_k10_kate_sso, digits = 4))
pd.Series(y_pred_k10_kate_sso).to_csv('llama2016_k10_kate_sso.csv', index=False)

In [ ]:
y_pred_k10_kate_sl = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='kate', dem_order = 'sl', k = 10, order_seed = 1, selection_seed = 1)
print('\n 10-shot kate sl: \n', classification_report(y_test, y_pred_k10_kate_sl, digits = 4))
pd.Series(y_pred_k10_kate_sl).to_csv('llama2016_k10_kate_sl.csv', index=False)

In [ ]:
y_pred_k10_kate_swn = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='kate', dem_order = 'swn', k = 10, order_seed = 1, selection_seed = 1)
print('\n 10-shot kate swn: \n', classification_report(y_test, y_pred_k10_kate_swn, digits = 4))
pd.Series(y_pred_k10_kate_swn).to_csv('llama2016_k10_kate_swn.csv', index=False)

In [ ]:
y_pred_k10_kate_ce = run_few_shot_preds(dataset_train, dataset_test, dem_selection ='kate', dem_order = 'ce', k = 10, order_seed = 1, selection_seed = 1)
print('\n 10-shot kate ce: \n', classification_report(y_test, y_pred_k10_kate_ce, digits = 4))
pd.Series(y_pred_k10_kate_ce).to_csv('llama2016_k10_kate_ce.csv', index=False)